# Option 1 — Cepstrum (Male/Female)
This notebook implements the required flow:
1) Load and plot original signals
2) Apply Hamming window
3) High-pass + Low-pass filtering (band-pass)
4) FFT + spectra
5) log(|spectrum|)
6) IFFT → real cepstrum
7) Run on test files and compute cosine similarity + visualization

Upload the WAV files (male.wav, female.wav, male_4_test.wav, female_4_test.wav) to the same folder as this notebook, or update paths below.

In [ ]:
# If running in Google Colab:
# from google.colab import files
# files.upload()

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile
from scipy import signal
from scipy.fft import rfft, rfftfreq, irfft

from pathlib import Path


In [ ]:
# ---- Paths (edit if needed) ----
# In the sandbox environment used to create this notebook, files were at /mnt/data/...
# In Colab, you typically put them under /content/...
BASE_DIR = Path(".")  # change to Path("/content") in Colab if needed

FILES = {
    "male_ref": BASE_DIR / "male.wav",
    "female_ref": BASE_DIR / "female.wav",
    "male_test": BASE_DIR / "male_4_test.wav",
    "female_test": BASE_DIR / "female_4_test.wav",
}

FILES


In [ ]:
def load_wav_mono(path: Path):
    sr, x = wavfile.read(str(path))
    # Convert to float32 in [-1, 1]
    if x.dtype.kind in ("i", "u"):
        maxv = np.iinfo(x.dtype).max
        x = x.astype(np.float32) / maxv
    else:
        x = x.astype(np.float32)

    # If stereo, average to mono
    if x.ndim == 2:
        x = x.mean(axis=1)

    return sr, x

def plot_time(sr, x, title):
    t = np.arange(len(x)) / sr
    plt.figure()
    plt.plot(t, x)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.grid(True)
    plt.show()

def hamming_window(x):
    w = np.hamming(len(x)).astype(np.float32)
    return x * w, w

def bandpass_filter(sr, x, hp_hz=80.0, lp_hz=8000.0, order=4):
    # Butterworth high-pass and low-pass (applied as band-pass)
    nyq = 0.5 * sr
    hp = max(1.0, float(hp_hz))
    lp = min(0.45 * sr, float(lp_hz))  # keep below Nyquist

    b_hp, a_hp = signal.butter(order, hp / nyq, btype="highpass")
    b_lp, a_lp = signal.butter(order, lp / nyq, btype="lowpass")

    y = signal.filtfilt(b_hp, a_hp, x)
    y = signal.filtfilt(b_lp, a_lp, y)
    return y

def compute_spectrum(sr, x):
    # Real FFT
    X = rfft(x)
    f = rfftfreq(len(x), d=1.0/sr)
    mag = np.abs(X)
    return f, X, mag

def plot_spectrum(f, mag, title, max_freq=None):
    plt.figure()
    if max_freq is not None:
        idx = f <= max_freq
        plt.plot(f[idx], mag[idx])
    else:
        plt.plot(f, mag)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("|X(f)|")
    plt.title(title)
    plt.grid(True)
    plt.show()

def log_magnitude(mag, eps=1e-12):
    return np.log(mag + eps)

def real_cepstrum_from_logmag(logmag):
    # Cepstrum = IFFT(log |FFT(x)|). Using irfft because logmag is on rfft bins.
    c = irfft(logmag)
    return c

def plot_cepstrum(sr, c, title, max_quef_ms=50):
    # quefrency axis in ms
    q = (np.arange(len(c)) / sr) * 1000.0
    plt.figure()
    idx = q <= max_quef_ms
    plt.plot(q[idx], c[idx])
    plt.xlabel("Quefrency (ms)")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.grid(True)
    plt.show()

def cosine_similarity(a, b, eps=1e-12):
    a = np.asarray(a).ravel()
    b = np.asarray(b).ravel()
    # Make equal length
    n = min(len(a), len(b))
    a = a[:n]
    b = b[:n]
    return float(np.dot(a, b) / ((np.linalg.norm(a) * np.linalg.norm(b)) + eps))


## A. Load and plot original signals

In [ ]:
sr_m, x_m = load_wav_mono(FILES["male_ref"])
sr_f, x_f = load_wav_mono(FILES["female_ref"])

assert sr_m == sr_f, f"Sample rates differ: male={sr_m}, female={sr_f}"
sr = sr_m

plot_time(sr, x_m, "Male — original waveform")
plot_time(sr, x_f, "Female — original waveform")


## B.1 Hamming window

In [ ]:
x_m_win, w_m = hamming_window(x_m)
x_f_win, w_f = hamming_window(x_f)

# Show window shape (they are identical if length equal)
plt.figure()
plt.plot(w_m)
plt.title("Hamming window")
plt.xlabel("Sample index")
plt.ylabel("w[n]")
plt.grid(True)
plt.show()

plot_time(sr, x_m_win, "Male — after Hamming window")
plot_time(sr, x_f_win, "Female — after Hamming window")


## B.2 Noise reduction with High-pass + Low-pass filters (band-pass)

In [ ]:
# Typical speech-focused band: ~80Hz–8000Hz (you may adjust)
x_m_filt = bandpass_filter(sr, x_m_win, hp_hz=80, lp_hz=8000, order=4)
x_f_filt = bandpass_filter(sr, x_f_win, hp_hz=80, lp_hz=8000, order=4)

plot_time(sr, x_m_filt, "Male — after HP+LP filtering")
plot_time(sr, x_f_filt, "Female — after HP+LP filtering")


## B.3 FFT and spectra

In [ ]:
f_m, X_m, mag_m = compute_spectrum(sr, x_m_filt)
f_f, X_f, mag_f = compute_spectrum(sr, x_f_filt)

plot_spectrum(f_m, mag_m, "Male — magnitude spectrum", max_freq=8000)
plot_spectrum(f_f, mag_f, "Female — magnitude spectrum", max_freq=8000)


## C. log(|spectrum|)

In [ ]:
logmag_m = log_magnitude(mag_m)
logmag_f = log_magnitude(mag_f)

plt.figure()
plt.plot(f_m[f_m <= 8000], logmag_m[f_m <= 8000])
plt.title("Male — log magnitude spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("log(|X(f)|)")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(f_f[f_f <= 8000], logmag_f[f_f <= 8000])
plt.title("Female — log magnitude spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("log(|X(f)|)")
plt.grid(True)
plt.show()


## D. IFFT (real cepstrum)

In [ ]:
c_m = real_cepstrum_from_logmag(logmag_m)
c_f = real_cepstrum_from_logmag(logmag_f)

plot_cepstrum(sr, c_m, "Male — real cepstrum (IFFT of logmag)", max_quef_ms=50)
plot_cepstrum(sr, c_f, "Female — real cepstrum (IFFT of logmag)", max_quef_ms=50)


## Helper: Full pipeline returning cepstrum feature vector

In [ ]:
def cepstrum_pipeline(path: Path, hp_hz=80, lp_hz=8000, max_quef_ms=50):
    sr, x = load_wav_mono(path)
    x_win, _ = hamming_window(x)
    x_filt = bandpass_filter(sr, x_win, hp_hz=hp_hz, lp_hz=lp_hz, order=4)
    f, X, mag = compute_spectrum(sr, x_filt)
    logmag = log_magnitude(mag)
    c = real_cepstrum_from_logmag(logmag)

    # Use only a short quefrency region as a feature (common for pitch/formant cues)
    q_ms = (np.arange(len(c)) / sr) * 1000.0
    feat = c[q_ms <= max_quef_ms].copy()
    return sr, x, x_win, x_filt, f, mag, logmag, c, feat


## E. Run pipeline on test files + cosine similarity + visualization

In [ ]:
# Extract feature vectors
_, *_ , feat_m_ref = cepstrum_pipeline(FILES["male_ref"])
_, *_ , feat_f_ref = cepstrum_pipeline(FILES["female_ref"])

sr_mt, x_mt, x_mt_win, x_mt_filt, f_mt, mag_mt, logmag_mt, c_mt, feat_m_test = cepstrum_pipeline(FILES["male_test"])
sr_ft, x_ft, x_ft_win, x_ft_filt, f_ft, mag_ft, logmag_ft, c_ft, feat_f_test = cepstrum_pipeline(FILES["female_test"])

# Cosine similarities
sim_mtest_to_m = cosine_similarity(feat_m_test, feat_m_ref)
sim_mtest_to_f = cosine_similarity(feat_m_test, feat_f_ref)

sim_ftest_to_m = cosine_similarity(feat_f_test, feat_m_ref)
sim_ftest_to_f = cosine_similarity(feat_f_test, feat_f_ref)

print("Male TEST similarity:")
print(f"  to Male REF:   {sim_mtest_to_m:.4f}")
print(f"  to Female REF: {sim_mtest_to_f:.4f}")

print("\nFemale TEST similarity:")
print(f"  to Male REF:   {sim_ftest_to_m:.4f}")
print(f"  to Female REF: {sim_ftest_to_f:.4f}")


In [ ]:
# Simple visualization: bar charts
labels = ["to Male REF", "to Female REF"]

plt.figure()
plt.bar(labels, [sim_mtest_to_m, sim_mtest_to_f])
plt.title("Cosine similarity — Male TEST")
plt.ylabel("Similarity")
plt.grid(True, axis="y")
plt.show()

plt.figure()
plt.bar(labels, [sim_ftest_to_m, sim_ftest_to_f])
plt.title("Cosine similarity — Female TEST")
plt.ylabel("Similarity")
plt.grid(True, axis="y")
plt.show()


In [ ]:
# Optional: visualize test cepstra vs references (same quefrency range)
def plot_cepstra_compare(sr, feat_test, feat_ref, title, max_quef_ms=50):
    q = (np.arange(len(feat_test)) / sr) * 1000.0
    plt.figure()
    plt.plot(q, feat_test, label="test")
    plt.plot(q, feat_ref[:len(feat_test)], label="reference", alpha=0.8)
    plt.xlabel("Quefrency (ms)")
    plt.ylabel("Cepstrum amplitude")
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

# Align sample rates assumption for quefrency axis
plot_cepstra_compare(sr, feat_m_test, feat_m_ref, "Male TEST vs Male REF (cepstrum features)")
plot_cepstra_compare(sr, feat_m_test, feat_f_ref, "Male TEST vs Female REF (cepstrum features)")

plot_cepstra_compare(sr, feat_f_test, feat_f_ref, "Female TEST vs Female REF (cepstrum features)")
plot_cepstra_compare(sr, feat_f_test, feat_m_ref, "Female TEST vs Male REF (cepstrum features)")
